In [1]:
import pandas as pd
import numpy as np
import re 
import plotly.graph_objects as go
import math
import scipy.stats as stat

In [2]:
df = pd.read_excel('/Users/lucaslevassor/projects/Signal_peptide_project/data/03_proteomics_data/annotated_20221124_FJ_E1200_MWN_15cm_140min_500ng_#1572_proteins.xlsx')

In [3]:
df

,Checked,Protein FDR Confidence: Combined,Master,Accession,Description,Exp. q-value: Combined,Sum PEP Score,Coverage [%],# Peptides,# PSMs,...,"Found in Sample: [S23] F23: Sample, 8","Found in Sample: [S24] F24: Sample, 9","Found in Sample: [S25] F25: Sample, 10","Found in Sample: [S26] F26: Sample, 11","Found in Sample: [S27] F27: Sample, 12","Found in Sample: [S28] F28: Sample, 13","Found in Sample: [S29] F29: Sample, 14","Found in Sample: [S30] F30: Sample, 15",# Protein Groups,Modifications
0,False,High,Master Protein,AO090003000935-T-p1,transcript=AO090003000935-T | gene=AO090003000...,0.000,599.687,89,31,7127,...,High,High,High,High,High,High,High,High,1,NaN
1,False,High,Master Protein,AO090023000944-T-p1,transcript=AO090023000944-T | gene=AO090023000...,0.000,523.784,68,32,3237,...,High,High,High,Peak Found,Peak Found,High,High,High,1,NaN
2,False,High,Master Protein,AO090003001591-T-p1,transcript=AO090003001591-T | gene=AO090003001...,0.000,519.742,68,33,2987,...,High,High,Not Found,Not Found,Not Found,High,High,High,1,NaN
3,False,High,Master Protein,RFP_Fusion,RFP_Fusion,0.000,450.464,64,35,398,...,High,Peak Found,High,Peak Found,High,High,High,High,1,NaN
4,False,High,Master Protein,AO090005001300-T-p1,transcript=AO090005001300-T | gene=AO090005001...,0.000,384.472,90,45,554,...,High,High,High,High,High,High,High,High,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,False,Medium,Master Protein,AO090001000075-T-p1,transcript=AO090001000075-T | gene=AO090001000...,0.044,1.795,4,1,1,...,Not Found,Peak Found,Peak Found,Peak Found,Peak Found,Peak Found,Peak Found,Peak Found,1,NaN
862,False,Medium,Master Protein,AO090005001355-T-p1,transcript=AO090005001355-T | gene=AO090005001...,0.046,1.790,14,1,1,...,Peak Found,Peak Found,Peak Found,Not Found,Peak Found,Not Found,Peak Found,Not Found,1,NaN
863,False,Medium,Master Protein,AO090003000247-T-p1,transcript=AO090003000247-T | gene=AO090003000...,0.047,1.779,4,1,1,...,Not Found,Peak Found,Peak Found,Peak Found,Peak Found,Not Found,Not Found,Not Found,1,Met-loss [N-Term]
864,False,Medium,Master Protein,AO090001000680-T-p1,transcript=AO090001000680-T | gene=AO090001000...,0.048,1.774,1,1,1,...,Not Found,Not Found,Peak Found,Peak Found,Not Found,Peak Found,High,Peak Found,1,NaN


In [4]:
acc_numbers = df['Accession'].to_list()

In [5]:
ory7_mm = df.iloc[: , 55:58]

damyR = df.iloc[: , 58:61]

dprtT = df.iloc[: , 61:64]

double_KO = df.iloc[: , 64:67]

ory7_AM = df.iloc[: , 67:70]

all_strains_df = [ory7_mm, damyR, dprtT, double_KO, ory7_AM]

Calculating the mean

In [6]:
means = []
for df in all_strains_df: 
    means.append(df.mean(axis=1))

In [7]:
ory7_mm

,ORY_MM1,ORY_MM2,ORY_MM3
0,9.597858e+07,9.416572e+07,1.297344e+08
1,2.378266e+08,3.016177e+08,2.504219e+08
2,9.888853e+09,1.123887e+10,1.030518e+10
3,6.556024e+05,8.538809e+05,5.388381e+05
4,7.204276e+07,3.532077e+07,1.257048e+07
...,...,...,...
861,1.819305e+05,2.070978e+04,NaN
862,2.104682e+04,5.864526e+04,NaN
863,NaN,NaN,NaN
864,NaN,NaN,NaN


In [8]:
foldchanges_ory7_mm_vs_damyR = list(np.log2(np.divide(means[1], means[0])))
foldchanges_ory7_mm_vs_dprtT = list(np.log2(np.divide(means[2], means[0])))


Calculating p-values

In [9]:
ory7_mm.shape[0]

866

In [10]:
def pvalues(df1, df2):
    pvals = []
    num_values = df1.shape[0]
    for row in range(0, num_values):
        ttest_result = stat.ttest_ind(df1.iloc[row, :],  df2.iloc[row, :])
        pvalue = ttest_result[1]
        pvals.append(pvalue)
    return pvals


pval_ory7_mm_vs_damyR = pvalues(ory7_mm, damyR)
pval_ory7_mm_vs_dprtT = pvalues(ory7_mm, damyR)

Bonferroni correction 

In [11]:
num_values = ory7_mm.shape[0]

transformed_pvals_ory7_mm_vs_damyR = list(-1*np.log10(num_values*np.array(pval_ory7_mm_vs_damyR)))
transformed_pvals_ory7_mm_vs_dprtT= list(-1*np.log10(num_values*np.array(pval_ory7_mm_vs_dprtT)))


In [27]:
import plotly.graph_objects as go

def plot_volcano(foldchanges:list, transformed_pvals:list, acc_numbers:list):
    plot_title = "Group 1 vs Group 2"
    x_axis_title = "log2 fold change"
    y_axis_title = "-log10 pvalue"
    point_radius = 8

    fig = go.Figure()

    # Define colors
    significant_upregulated = '#e63946'  # Red
    significant_downregulated = '#457b9d'  # Blue
    non_significant = '#969696'

    colors = []
    for fc in foldchanges:
        if fc > 2:
            colors.append(significant_upregulated)
        elif fc < -2:
            colors.append(significant_downregulated)
        else:
            colors.append(non_significant)

    # Scatter plot
    fig.add_trace(
        go.Scattergl(
            x=foldchanges,
            y=transformed_pvals,
            mode='markers',
            text=acc_numbers,
            hoverinfo='text+x+y',
            hovertemplate='<b>%{text}</b><br>%{xaxis.title.text}: %{x}<br>%{yaxis.title.text}: %{y}',
            marker={
                'color': colors,
                'size': point_radius,
                'line': {
                    'width': 0.5,
                    'color': 'rgba(0, 0, 0, 0.5)'
                }
            }
        )
    )
    
    # Vertical lines at x=-2 and x=2
    fig.add_shape(
        type="line",
        x0=-2,
        x1=-2,
        y0=min(transformed_pvals),
        y1=max(transformed_pvals),
        line=dict(
            color="rgba(0, 0, 0, 0.5)",
            width=2
        )
    )

    fig.add_shape(
        type="line",
        x0=2,
        x1=2,
        y0=min(transformed_pvals),
        y1=max(transformed_pvals),
        line=dict(
            color="rgba(0, 0, 0, 0.5)",
            width=2
        )
    )

    # Layout
    fig.update_layout(
        title=plot_title,
        xaxis_title=x_axis_title,
        yaxis_title=y_axis_title,
        paper_bgcolor='white',
        plot_bgcolor='white',
        xaxis=dict(
            gridcolor='rgba(200,200,200,0.5)',
            zeroline=False,   # This line removes the horizontal zero line
            showgrid=True,
            titlefont=dict(size=14),
            tickfont=dict(size=12)
        ),
        yaxis=dict(
            gridcolor='rgba(200,200,200,0.5)',
            zerolinecolor='rgba(150,150,150,0.7)',
            showgrid=True,
            titlefont=dict(size=14),
            tickfont=dict(size=12)
        ),
        hoverlabel=dict(
            bgcolor='white',
            font_size=13,
            font_family="Rockwell"
        ),
        legend_title_text='Significance',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    fig.show()


## Ory7 vs damyR

In [28]:
plot_volcano(foldchanges_ory7_mm_vs_damyR, transformed_pvals_ory7_mm_vs_damyR, acc_numbers)

In [24]:
ory7_mm_vs_damyR_df = pd.DataFrame({'acc':acc_numbers, 'foldchanges': foldchanges_ory7_mm_vs_damyR, 'transformed_pvals':transformed_pvals_ory7_mm_vs_damyR})
ory7_mm_vs_damyR_df = ory7_mm_vs_damyR_df.sort_values(by='foldchanges', ascending=False)
# Top50
ory7_mm_vs_damyR_df.head(50)


,acc,foldchanges,transformed_pvals
440,AO090102000579-T-p1,6.519062,NaN
840,AO090023000946-T-p1,5.258979,NaN
9,AO090003001036-T-p1,5.067028,0.171760
0,AO090003000935-T-p1,4.960833,2.113454
536,AO090701000512-T-p1,4.827687,-0.452659
465,AO090011000596-T-p1,4.788638,NaN
136,AO090005000588-T-p1,4.775131,-1.955153
156,AO090005000639-T-p1,4.720329,-0.830173
293,AO090124000044-T-p1,4.692341,-1.135243
24,AO090020000521-T-p1,4.484357,1.245072


In [25]:
# without NaN values
ory7_mm_vs_damyR_df.dropna().head(50)


,acc,foldchanges,transformed_pvals
9,AO090003001036-T-p1,5.067028,0.171760
0,AO090003000935-T-p1,4.960833,2.113454
536,AO090701000512-T-p1,4.827687,-0.452659
136,AO090005000588-T-p1,4.775131,-1.955153
156,AO090005000639-T-p1,4.720329,-0.830173
293,AO090124000044-T-p1,4.692341,-1.135243
24,AO090020000521-T-p1,4.484357,1.245072
29,AO090103000076-T-p1,4.373901,1.452133
127,AO090102000089-T-p1,4.231216,-1.887149
87,AO090102000558-T-p1,4.210320,-1.082903


## Ory7 vs dprtT

In [26]:
plot_volcano(foldchanges_ory7_mm_vs_dprtT, transformed_pvals_ory7_mm_vs_dprtT, acc_numbers)


In [16]:
ory7_mm_vs_dprT_df = pd.DataFrame({'acc':acc_numbers, 'foldchanges': foldchanges_ory7_mm_vs_dprtT, 'transformed_pvals':transformed_pvals_ory7_mm_vs_dprtT})
ory7_mm_vs_dprT_df = ory7_mm_vs_dprT_df.sort_values(by='foldchanges', ascending=False)
# Top50
ory7_mm_vs_dprT_df.head(50)


,acc,foldchanges,transformed_pvals
161,AO090010000704-T-p1,8.678198,NaN
155,AO090011000414-T-p1,7.786124,NaN
176,AO090003000661-T-p1,7.315601,NaN
171,AO090011000195-T-p1,6.625986,-0.838856
113,AO090009000634-T-p1,6.620568,NaN
287,AO090026000264-T-p1,6.473285,NaN
138,AO090011000202-T-p1,6.235646,NaN
164,AO090012000809-T-p1,6.130294,NaN
218,AO090005000996-T-p1,6.121530,NaN
143,AO090011000626-T-p1,6.032921,NaN


In [19]:
# without NaN values
ory7_mm_vs_dprT_df.dropna().head(50)


,acc,foldchanges,transformed_pvals
171,AO090011000195-T-p1,6.625986,-0.838856
154,AO090003001322-T-p1,5.611993,-2.573213
153,AO090011000053-T-p1,5.249946,-2.856280
45,AO090038000395-T-p1,5.149528,-2.258398
129,AO090023000923-T-p1,4.734847,-2.309280
82,AO090010000482-T-p1,4.575280,-2.896213
136,AO090005000588-T-p1,4.458283,-1.955153
75,AO090010000729-T-p1,4.387797,-2.263367
412,AO090001000218-T-p1,4.353718,-2.295923
13,AO090003000055-T-p1,4.323649,-2.247332
